In [1]:
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import root_mean_squared_error

In [ ]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")


In [4]:
data_path = "C:\\Users\\Marcelo\\Documents\\mlops-zoomcamp\\data"

In [5]:
df = pd.read_parquet(f'{data_path}\\green_tripdata_2021-01.parquet')
df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [ ]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

In [7]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
train_data_path = f'{data_path}\\green_tripdata_2021-01.parquet'
val_data_path = f'{data_path}\\green_tripdata_2021-02.parquet'

In [9]:
df_train = read_dataframe(train_data_path)
df_val = read_dataframe(val_data_path)

In [ ]:
len(df_train),len(df_val)

In [11]:
train_dicts = df[categorical+numerical].to_dict(orient='records')

In [12]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [13]:
target = 'duration'
y_train = df[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)

In [15]:
y_pred = lr.predict(X_train)

In [ ]:
#plt.figure(figsize=(10, 6))
sns.kdeplot(data=y_pred, label='prediction')
sns.kdeplot(data=y_train, label='actual')
plt.legend()
plt.show()

In [ ]:
root_mean_squared_error(y_train,y_pred)

In [18]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [19]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [20]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [35]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [34]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [36]:
from sklearn.metrics import mean_squared_error
import numpy as np

In [24]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42,
    # Parâmetros específicos para GPU
    'device': 'gpu',  # Usa GPU para construção de árvores
    #'gpu_id': 0,  # ID da GPU a ser usada (0 para primeira GPU)
    #'predictor': 'gpu_predictor'  # Usa GPU para predição
}

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost-gpu")  # Adicionei -gpu para identificar que está usando GPU
        mlflow.log_params(params)
        
        # Configuração do treinamento
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        mse = mean_squared_error(y_val, y_pred) # Calcula o MSE
        rmse = np.sqrt(mse) # Calcula a raiz quadrada (RMSE)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

In [40]:
config = {
        "device": "gpu",
        "learning_rate": 0.11536248369973928,
        "max_depth": 45,
        "min_child_weight": 1.152401001199661,
        "objective": "reg:squarederror",
        "reg_alpha": 0.05317357113372023,
        "reg_lambda": 0.061055701274306054,
        "seed": 42
    }
def train_model_xgboost(config,train,valid,num_boost_round=1000):
    mlflow.xgboost.autolog()
    booster = xgb.train(
                params=config,
                dtrain=train,
                num_boost_round=num_boost_round,
                evals=[(valid, 'validation')],
                early_stopping_rounds=50
            )
    return booster

In [31]:
import pickle

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("developer", "marcelo")
    mlflow.log_param("train-data-path", train_data_path)
    mlflow.log_param("valid-data-path", val_data_path)
    alpha = 0.001
    mlflow.log_param("model", "lasso")
    lr = Lasso(alpha=alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metrics({"rmse": rmse})
    mlflow.log_params({"alpha": alpha})
    mlflow.log_artifact(local_path="..\\01-intro\\models\\lr_regressor.bin",
                        artifact_path="models_pickle")

In [38]:
mlflow.xgboost.autolog(disable=True)

In [41]:
with mlflow.start_run():
    mlflow.log_params(config)
    booster = train_model_xgboost(config,train,valid,num_boost_round=200)
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")
    with open("..\\01-intro\\models\\preprocessor.bin", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("..\\01-intro\\models\\preprocessor.bin",
                        artifact_path="preprocessor")


2025/05/26 15:24:59 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.4.2 <= xgboost <= 3.0.0, but the installed version is 3.0.2. If you encounter errors during autologging, try upgrading / downgrading xgboost to a compatible version, or try upgrading MLflow.


[0]	validation-rmse:11.28151
[1]	validation-rmse:10.48816
[2]	validation-rmse:9.81741
[3]	validation-rmse:9.25049
[4]	validation-rmse:8.77726
[5]	validation-rmse:8.38408
[6]	validation-rmse:8.05609
[7]	validation-rmse:7.78492
[8]	validation-rmse:7.56104
[9]	validation-rmse:7.37649
[10]	validation-rmse:7.22330
[11]	validation-rmse:7.10060
[12]	validation-rmse:6.99730
[13]	validation-rmse:6.91438
[14]	validation-rmse:6.84376
[15]	validation-rmse:6.78708
[16]	validation-rmse:6.73830
[17]	validation-rmse:6.69749
[18]	validation-rmse:6.66438
[19]	validation-rmse:6.63507
[20]	validation-rmse:6.61044
[21]	validation-rmse:6.58872
[22]	validation-rmse:6.57103
[23]	validation-rmse:6.55625
[24]	validation-rmse:6.54279
[25]	validation-rmse:6.53134
[26]	validation-rmse:6.52110
[27]	validation-rmse:6.51254
[28]	validation-rmse:6.50488
[29]	validation-rmse:6.49750
[30]	validation-rmse:6.49165
[31]	validation-rmse:6.48585
[32]	validation-rmse:6.48110
[33]	validation-rmse:6.47675
[34]	validation-rmse:6

2025/05/26 15:26:40 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/05/26 15:26:40 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Marcelo\AppData\Local\anaconda3\envs\environment\lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [15:26:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/05/26 15:26:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
c:\Users\Marcelo\AppData\Local\anaconda3\envs\environment\lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [15:26:46] WARNING: C:\actions-runner\_wo

In [ ]:
y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

In [ ]:
with open('models/lr_regressor.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)